# Neural network architecture

Dingo is based on a method called [Neural posterior estimation](https://arxiv.org/abs/1605.06376), see [here](sbi.md) for an introduction. A central object is the conditional neural density estimator, a deep neural network trained to represent the Bayesian posterior. This section describes the neural network architecture developed in {cite:p}`Dax:2021tsq`, and subsequently used in {cite:p}`Dax:2021myb`, {cite:p}`Dax:2022pxd` and {cite:p}`Wildberger:2022agw`. Note that Dingo can easily be extended to different architectures.

## Neural spline flow with SVD compression
The architecture consists of two compenents, the embedding network which compresses the high-dimensionl data to a lower dimensional feature vector, and the conditional normalizing flow which estimates the Bayesian posterior based on this feature vector. Both components are trained jointly and end-to-end with the objective descriped [here](sbi.md). The network can be build with

In [29]:
from dingo.core.nn.nsf import create_nsf_with_rb_projection_embedding_net

### Embedding network
The embedding network compresses the high-dimensional conditioning information (consisting of frequency domain strain and PSD data). The first layer of this network is initialized with an [SVD](https://en.wikipedia.org/wiki/Singular_value_decomposition) matrix from a reduced basis built with non-noisy waveforms. This projection filters out the noise that is orthogonal to the signal manifold, and significantly simplifies the task for the neural network.

The initial compression layer is followed by a sequence of residual blocks consisting of dense layers for further compression. Example kwargs:

In [ ]:
embedding_net_kwargs = {
    "input_dims": (2, 3, 8033),
    "output_dim": 128,
    "hidden_dims": [
        1024, 1024, 1024, 1024, 1024, 1024, \
        512, 512, 512, 512, 512, 512, \
        256, 256, 256, 256, 256, 256, \
        128, 128, 128, 128, 128, 128
    ],
    "activation": "elu",
    "dropout": 0.0,
    "batch_norm": True,
    "svd": {
        "num_training_samples": 50000,
        "num_validation_samples": 5000,
        "size": 200,
    }
}

Here, `input_dims=(2, 3, 8033)` refers to the input dimension, for frequency domain data with `8033` frequency bins and `3` channels (real part, complex part, ASD) in `2` detectors. The embedding network compresses this to `output_dim=128` components. The SVD initialization is controlled with the `svd` argument, and the residual blocks are specified with `hidden_dims`.

```{note}
Not all of these arguments have to be set in the configuration file when training dingo. For example, the `input_dims` argument is automatically filled in based on the specified domain information and number of detectors. Similarly, the `context_dim` of the flow (see below) is filled in based on the `output_dim` of the embedding network and the number of [GNPE](gnpe.md) proxies. See the [Dingo examples](https://github.com/dingo-gw/dingo/tree/main/examples) for the corresponding configuration files and training commands.
```

### Flow
We use the [neural spline flow](https://arxiv.org/abs/1906.04032) as a density estimator. This takes the output of the embedding network as context information and estimates the Bayesian posterior distribution. Example kwargs:

In [25]:
nsf_kwargs = {
    "input_dim": 15,
    "context_dim": 129,
    "num_flow_steps": 30,
    "base_transform_kwargs": {
        "hidden_dim": 512,
        "num_transform_blocks": 5,
        "activation": "elu",
        "dropout_probability": 0.0,
        "batch_norm": True,
        "num_bins": 8,
        "base_transform_type": "rq-coupling",
    },
}

This creates a neural spline flow with `input_dim=15` parameters, conditioned on a `129` dimensional context vector, corresponding to the `128` dimensional output of the embedding network and one [GNPE](gnpe.md) proxy variable. The neural spline flow consists of `num_flow_steps=30` layers, for which the transformation is specified with `base_transform_kwargs`.

In [33]:
create_nsf_with_rb_projection_embedding_net(nsf_kwargs, embedding_net_kwargs)

FlowWrapper(
  (embedding_net): Sequential(
    (0): LinearProjectionRB(
      (layers_rb): ModuleList(
        (0): Linear(in_features=24099, out_features=400, bias=True)
        (1): Linear(in_features=24099, out_features=400, bias=True)
      )
    )
    (1): DenseResidualNet(
      (initial_layer): Linear(in_features=800, out_features=1024, bias=True)
      (blocks): ModuleList(
        (0): ResidualBlock(
          (batch_norm_layers): ModuleList(
            (0): BatchNorm1d(1024, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (1): BatchNorm1d(1024, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          )
          (linear_layers): ModuleList(
            (0): Linear(in_features=1024, out_features=1024, bias=True)
            (1): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (1): ResidualBlock(
          (batch_norm_layers): ModuleList(
      